In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
from shapely.geometry import Polygon
import pickle

Key meta data

In [2]:
root_path = "D:/GeoData/"
Main_CRS = "EPSG:27700"

Shapefiles

In [3]:
Shape_Loc = {
    'All_GB' : root_path + "Countries__December_2019__Boundaries_UK_BFC-shp/Countries__December_2019__Boundaries_UK_BFC.shp",
    'National_Parks' : root_path + "National_Parks__December_2019__GB_BFE-shp/National_Parks__December_2019__GB_BFE.shp",
    'LocalAuthorities' : root_path + "Local_Authority_Districts__May_2020__Boundaries_UK_BFC-shp/Local_Authority_Districts__May_2020__Boundaries_UK_BFC.shp",
    'LSOA' : root_path + "Lower_Layer_Super_Output_Area__December_2011__EW_BSC_V2-shp/Lower_Layer_Super_Output_Area__December_2011__EW_BSC_V2.shp",
    'GreenSpace' : root_path + "opgrsp_essh_gb/OS Open Greenspace (ESRI Shape File) GB/data/GB_GreenspaceSite.shp",
    'Rivers' : root_path + "oprvrs_essh_gb/data/WatercourseLink.shp",
    'Railway_Lines' : root_path + "strtgi_essh_gb/data/railway_line.shp",
    'Woodland_Region' : root_path + "strtgi_essh_gb/data/woodland_region.shp",
    'Urban_Region' : root_path + "strtgi_essh_gb/data/urban_region.shp",
    'Foreshor_Region' : root_path + "strtgi_essh_gb/data/foreshor_region.shp",
    'Ferry_Line' : root_path + "strtgi_essh_gb/data/ferry_line.shp",
    'Coastline' : root_path + "strtgi_essh_gb/data/coastline.shp" ,
    'Gridlines' : root_path + "strtgi_essh_gb/data/gridlines.shp",
    'Lakes' : root_path + "strtgi_essh_gb/data/lakes_region.shp"}

In [4]:
All_Maps = dict((k, gpd.read_file(v).to_crs(Main_CRS)) for k, v in Shape_Loc.items())

In [5]:
file = os.listdir("D:\GeoData\oproad_essh_gb\data")

path = [os.path.join("D:\GeoData\oproad_essh_gb\data", i) for i in file if "RoadLink.shp" in i]

All_Maps['Road_Link'] = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in path], 
                        ignore_index=True), crs=Main_CRS)

In [6]:
path = [os.path.join("D:\GeoData\oproad_essh_gb\data", i) for i in file if "MotorwayJunction.shp" in i]

All_Maps['MotorwayJunctions'] = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in path], 
                        ignore_index=True), crs=Main_CRS)

In [7]:
All_Maps['Motorway'] = All_Maps['Road_Link'][All_Maps['Road_Link']['class'] == "Motorway"]
All_Maps['A_Roads'] = All_Maps['Road_Link'][All_Maps['Road_Link']['class'] == "A Road"]
All_Maps['B_Roads'] = All_Maps['Road_Link'][All_Maps['Road_Link']['class'] == "B Road"]
All_Maps['Other_roads'] = All_Maps['Road_Link'][~All_Maps['Road_Link']["class"].isin([ "Motorway", "A Road", "B Road"])]

#del All_Maps['Road_Link']

NSPL Data

* Extra complications here because GB is on the GB grid and NI is on the Irish grid. 

In [8]:
NSPL_Data = pd.read_pickle(root_path + 'WorkingData/' + 'NSPL_Data.pickle')

#Split the data into NI and GB
NI = NSPL_Data[NSPL_Data["Postcode_NS"].str.slice(stop=2)=="BT"]
GB = NSPL_Data[NSPL_Data["Postcode_NS"].str.slice(stop=2)!="BT"]

#read the GB data using the GB national grid
GB_gdf = gpd.GeoDataFrame(GB, geometry=gpd.points_from_xy(GB.oseast1m, GB.osnrth1m), crs=Main_CRS)

#read the NI data using the Irish grid
NI_gdf = gpd.GeoDataFrame(NI, geometry=gpd.points_from_xy(NI.oseast1m, NI.osnrth1m), crs="EPSG:29902")

#Change NI data to the Main_CRS
NI_gdf = NI_gdf.to_crs(Main_CRS)

#Re-combine now they are on the same coordinates
All_Maps['NSPL_gdf'] = gpd.GeoDataFrame( pd.concat( [GB_gdf, NI_gdf], ignore_index=True) )

del NSPL_Data, NI, GB, NI_gdf, GB_gdf

C:\Users\richa\Anaconda3\envs\LandReg_py\lib\site-packages\geopandas\array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)
C:\Users\richa\Anaconda3\envs\LandReg_py\lib\site-packages\geopandas\array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


Land Registry Data

In [9]:
LandReg = pd.read_pickle(root_path + 'WorkingData/' + 'LandReg.pickle')
All_Maps['LandReg_gdf'] = gpd.GeoDataFrame(LandReg, geometry=gpd.points_from_xy(LandReg.oseast1m, LandReg.osnrth1m), crs=Main_CRS)

del LandReg

C:\Users\richa\Anaconda3\envs\LandReg_py\lib\site-packages\geopandas\array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


NaPTAN Data

In [10]:
RailwayStations = pd.read_pickle(root_path + 'WorkingData/' + 'RailwayStations_Data.pickle')
FerryTerminals = pd.read_pickle(root_path + 'WorkingData/' + 'FerryTerminals_Data.pickle')
BusStops = pd.read_pickle(root_path + 'WorkingData/' + 'BusStops_Data.pickle')

All_Maps['RailwayStations'] = gpd.GeoDataFrame(RailwayStations, geometry=gpd.points_from_xy(RailwayStations.Easting, RailwayStations.Northing), crs=Main_CRS)
All_Maps['FerryTerminals'] = gpd.GeoDataFrame(FerryTerminals, geometry=gpd.points_from_xy(FerryTerminals.Easting, FerryTerminals.Northing), crs=Main_CRS)
All_Maps['BusStops'] = gpd.GeoDataFrame(BusStops, geometry=gpd.points_from_xy(BusStops.Easting, BusStops.Northing), crs=Main_CRS)

C:\Users\richa\Anaconda3\envs\LandReg_py\lib\site-packages\geopandas\array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)
C:\Users\richa\Anaconda3\envs\LandReg_py\lib\site-packages\geopandas\array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)
C:\Users\richa\Anaconda3\envs\LandReg_py\lib\site-packages\geopandas\array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


Road Accidents

In [11]:
Road_Accidents = pd.read_pickle(root_path + 'WorkingData/' + 'Road_Accidents_Data.pickle')

All_Maps['Road_Accidents'] = gpd.GeoDataFrame(Road_Accidents, geometry=gpd.points_from_xy(Road_Accidents.location_easting_osgr, Road_Accidents.location_northing_osgr), crs=Main_CRS)

C:\Users\richa\Anaconda3\envs\LandReg_py\lib\site-packages\geopandas\array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


Schools

In [12]:
Schools = pd.read_pickle(root_path + 'WorkingData/' + 'Schools_Data.pickle')

All_Maps['Schools'] = gpd.GeoDataFrame(Schools, geometry=gpd.points_from_xy(Schools.gridgb1e, Schools.gridgb1n), crs=Main_CRS)

C:\Users\richa\Anaconda3\envs\LandReg_py\lib\site-packages\geopandas\array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


Fire Stations

In [13]:
FireStations = pd.read_pickle(root_path + 'WorkingData/' + 'FireStations_Data.pickle')

All_Maps['FireStations'] = gpd.GeoDataFrame(FireStations, geometry=gpd.points_from_xy(FireStations.STATION_EASTING, FireStations.STATION_NORTHING), crs=Main_CRS)

C:\Users\richa\Anaconda3\envs\LandReg_py\lib\site-packages\geopandas\array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


UDPRNs

In [ ]:
UDPRN = pd.read_pickle(root_path + 'WorkingData/' + 'NSUL_Data.pickle')
UDPRN = UDPRN.loc[:,["uprn", "gridgb1e", "gridgb1n", "pcds"]]

All_Maps['UDPRN'] = gpd.GeoDataFrame(UDPRN, geometry=gpd.points_from_xy(UDPRN.gridgb1e, UDPRN.gridgb1n), crs=Main_CRS)

There is a lot of mapping data here. My computer isn't that powerful so we need to subset the data. I've chosen a list of points and the code below clips the data to squares around these points

Functions for clipping and saving the maps

In [ ]:
def MakePolygon(centre, sq_sz, inp_crs):
    mnx = centre[0]-(sq_sz/2)
    mny = centre[1]-(sq_sz/2)
    mxx = centre[0]+(sq_sz/2)
    mxy = centre[1]+(sq_sz/2)
    polygon =Polygon([(mnx, mny), (mnx,mxy), (mxx,mxy), (mxx,mny), (mnx, mny)])
    poly_gdf = gpd.GeoDataFrame([1], geometry=[polygon], crs=inp_crs)
    return poly_gdf

In [ ]:
def save_obj(name , obj, extra=""):
    with open(root_path + 'WorkingData/' + extra + name + '.pkl', 'wb+') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
def load_obj(name ):
    with open(root_path + 'WorkingData/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
def ClipAndSave(name, centre, sq_sz, crs, extra):
    print(name)
    poly = MakePolygon(centre, sq_sz, crs)
    maps = dict((k, gpd.clip(v, poly)) for k, v in All_Maps.items())
    save_obj(name , maps, extra)
    return maps

Clip and save the maps for specific locations. Save at 5km^2

In [ ]:
save_obj('All_Maps' , All_Maps)

In [ ]:
KeyLocations = {'Grasmere': [334692 , 506545],
                'South_London' : [535493, 171250],
                'Loch_Lomond' : [238450 , 690986],
                'Furness' : [326877 , 474372],
                'Croydon' : [532621 , 165313],
                'StPauls' : [532052 , 181145],
                'M25' : [503078 , 178200],
                'M4_junct' : [519377 , 178360],
                'M6_Junctions' : [357659 , 425462],
                'Kendal': [353086 , 492882],
                'Milford Haven': [190649 , 205747],
                'Birmingham':[409295 , 290319],
                'Euston':[529947 , 182697],
                'Southampton':[441934 , 111013],
                'Windermere': [341217 , 498029],
                'AvonMouth': [351730 , 178243]}

In [ ]:
ClippedMaps_5k = dict((k, ClipAndSave(k, v, 5000, Main_CRS, "Clipped5k_")) for k, v in KeyLocations.items())
save_obj('ClippedMaps_5k' , ClippedMaps_5k)